In [9]:
# Import the dependencies.
import pandas as pd
import numpy as np
import time

In [10]:
# Import the requests library.
import requests

# Import the API key.
from ..\config import weather_api_key

SyntaxError: unexpected character after line continuation character (Temp/ipykernel_1640/708496283.py, line 5)

In [ ]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

In [4]:
# Create a set of random latitude and longitude combinations. zip packs the data in pairs
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [5]:
# Add the latitudes and longitudes to a list. This command is unpacking the zip array into a tuple list, 
# you can only do this once as it then removes it from memory
coordinates = list(lat_lngs)

In [6]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

760

In [7]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city. city.replace(" ","+") takes out the spaces and concatonates the name together
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_description = city_weather["weather"][0]["description"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | maragogi
Processing Record 2 of Set 1 | chivay
Processing Record 3 of Set 1 | punta arenas
Processing Record 4 of Set 1 | tias
Processing Record 5 of Set 1 | yar-sale
Processing Record 6 of Set 1 | chokurdakh
Processing Record 7 of Set 1 | juegang
Processing Record 8 of Set 1 | georgetown
Processing Record 9 of Set 1 | toliary
City not found. Skipping...
Processing Record 10 of Set 1 | tiksi
Processing Record 11 of Set 1 | leh
Processing Record 12 of Set 1 | longyearbyen
Processing Record 13 of Set 1 | touros
Processing Record 14 of Set 1 | goryachegorsk
Processing Record 15 of Set 1 | rikitea
Processing Record 16 of Set 1 | lawton
Processing Record 17 of Set 1 | tuktoyaktuk
Processing Record 18 of Set 1 | ushuaia
Processing Record 19 of Set 1 | mataura
Processing Record 20 of Set 1 | kodiak
Processing Record 21 of Set 1 | togur
Processing Record 22 of Set 1 | yellowknife
Processing Record 23 of 

Processing Record 37 of Set 4 | senanga
Processing Record 38 of Set 4 | flinders
Processing Record 39 of Set 4 | fortuna
Processing Record 40 of Set 4 | kuusamo
Processing Record 41 of Set 4 | totness
Processing Record 42 of Set 4 | alofi
Processing Record 43 of Set 4 | ghatanji
Processing Record 44 of Set 4 | tanhacu
Processing Record 45 of Set 4 | bambous virieux
Processing Record 46 of Set 4 | bennettsville
Processing Record 47 of Set 4 | saskylakh
Processing Record 48 of Set 4 | amderma
City not found. Skipping...
Processing Record 49 of Set 4 | chatrapur
Processing Record 50 of Set 4 | henties bay
Processing Record 1 of Set 5 | upernavik
Processing Record 2 of Set 5 | le mans
Processing Record 3 of Set 5 | kaitangata
Processing Record 4 of Set 5 | ellensburg
Processing Record 5 of Set 5 | half moon bay
Processing Record 6 of Set 5 | altamira
Processing Record 7 of Set 5 | sorland
Processing Record 8 of Set 5 | bredasdorp
Processing Record 9 of Set 5 | sola
Processing Record 10 of 

Processing Record 24 of Set 8 | ginir
Processing Record 25 of Set 8 | presque isle
Processing Record 26 of Set 8 | illoqqortoormiut
City not found. Skipping...
Processing Record 27 of Set 8 | lagoa
Processing Record 28 of Set 8 | bhandaria
Processing Record 29 of Set 8 | mokhsogollokh
Processing Record 30 of Set 8 | ucluelet
Processing Record 31 of Set 8 | swan hill
Processing Record 32 of Set 8 | carnot
Processing Record 33 of Set 8 | vila
Processing Record 34 of Set 8 | huarmey
Processing Record 35 of Set 8 | virginia beach
Processing Record 36 of Set 8 | maniitsoq
Processing Record 37 of Set 8 | samusu
City not found. Skipping...
Processing Record 38 of Set 8 | clyde river
Processing Record 39 of Set 8 | sangar
Processing Record 40 of Set 8 | nuuk
Processing Record 41 of Set 8 | amurzet
Processing Record 42 of Set 8 | wetzlar
Processing Record 43 of Set 8 | rio cuarto
Processing Record 44 of Set 8 | fukue
Processing Record 45 of Set 8 | mecca
Processing Record 46 of Set 8 | kiruna
P

Processing Record 12 of Set 12 | kokopo
Processing Record 13 of Set 12 | ivanava
Processing Record 14 of Set 12 | uthal
Processing Record 15 of Set 12 | vestmanna
Processing Record 16 of Set 12 | tarudant
City not found. Skipping...
Processing Record 17 of Set 12 | znamenskoye
Processing Record 18 of Set 12 | kokkola
Processing Record 19 of Set 12 | ust-ishim
Processing Record 20 of Set 12 | malpe
Processing Record 21 of Set 12 | daitari
Processing Record 22 of Set 12 | cornwall
Processing Record 23 of Set 12 | nantucket
Processing Record 24 of Set 12 | kununurra
Processing Record 25 of Set 12 | bulgan
Processing Record 26 of Set 12 | sinkat
City not found. Skipping...
Processing Record 27 of Set 12 | sedelnikovo
City not found. Skipping...
Processing Record 28 of Set 12 | kouango
Processing Record 29 of Set 12 | naberera
Processing Record 30 of Set 12 | petropavlovsk-kamchatskiy
Processing Record 31 of Set 12 | tuatapere
Processing Record 32 of Set 12 | assomada
Processing Record 33 o

Processing Record 49 of Set 15 | neuquen
Processing Record 50 of Set 15 | okakarara
Processing Record 1 of Set 16 | ebeltoft
Processing Record 2 of Set 16 | kaset sombun
Processing Record 3 of Set 16 | falun
Processing Record 4 of Set 16 | namanga
Processing Record 5 of Set 16 | warqla
City not found. Skipping...
Processing Record 6 of Set 16 | tirat karmel
Processing Record 7 of Set 16 | padang
Processing Record 8 of Set 16 | dalvik
Processing Record 9 of Set 16 | jega
Processing Record 10 of Set 16 | alencon
-----------------------------
Data Retrieval Complete      
-----------------------------


In [160]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Longyearbyen,78.2186,15.6401,10.24,61,0,17.27,SJ,clear sky
1,Busselton,-33.6500,115.3333,54.45,80,10,8.19,AU,clear sky
2,Mahebourg,-20.4081,57.7000,77.29,94,40,5.75,MU,scattered clouds
3,Ushuaia,-54.8000,-68.3000,38.82,87,40,28.77,AR,light intensity drizzle
4,Mujiayingzi,42.1167,118.7833,27.61,79,100,3.65,CN,light snow
5,Sao Filipe,14.8961,-24.4956,71.47,73,6,9.28,CV,clear sky
6,Medicine Hat,50.0501,-110.6683,47.71,68,100,5.01,CA,overcast clouds
7,Lebu,-37.6167,-73.6500,57.07,90,77,10.85,CL,broken clouds
8,Hilo,19.7297,-155.0900,77.25,80,100,3.44,US,overcast clouds
9,Arraial Do Cabo,-22.9661,-42.0278,75.06,94,75,11.50,BR,broken clouds


In [161]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp","Humidity","Cloudiness","Wind Speed","Current Description"]
city_data_df=city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Longyearbyen,SJ,78.2186,15.6401,10.24,61,0,17.27,clear sky
1,Busselton,AU,-33.6500,115.3333,54.45,80,10,8.19,clear sky
2,Mahebourg,MU,-20.4081,57.7000,77.29,94,40,5.75,scattered clouds
3,Ushuaia,AR,-54.8000,-68.3000,38.82,87,40,28.77,light intensity drizzle
4,Mujiayingzi,CN,42.1167,118.7833,27.61,79,100,3.65,light snow
5,Sao Filipe,CV,14.8961,-24.4956,71.47,73,6,9.28,clear sky
6,Medicine Hat,CA,50.0501,-110.6683,47.71,68,100,5.01,overcast clouds
7,Lebu,CL,-37.6167,-73.6500,57.07,90,77,10.85,broken clouds
8,Hilo,US,19.7297,-155.0900,77.25,80,100,3.44,overcast clouds
9,Arraial Do Cabo,BR,-22.9661,-42.0278,75.06,94,75,11.50,broken clouds


In [ ]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_Databse.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")